This notebook was inspired by https://github.com/asmitakulkarni/PriceEstimator/blob/master/01_Data_Acquisition.ipynb work in Etsy. We made some modifications, but used some of her methods for cleaning.

In [172]:
import pandas as pd
import pickle
from glob import glob as glob
from copy import deepcopy as copy
import numpy as np

In [2]:
pkls_paths = glob('/home/winston/CS591/*.pkl')

In [3]:
pkls_paths

['/home/winston/CS591/piercing.pkl',
 '/home/winston/CS591/pendant.pkl',
 '/home/winston/CS591/bracelet.pkl',
 '/home/winston/CS591/earrings.pkl',
 '/home/winston/CS591/necklace.pkl',
 '/home/winston/CS591/brooch.pkl',
 '/home/winston/CS591/ring.pkl',
 '/home/winston/CS591/anklet.pkl']

In [4]:
dfs = []
names = []

In [5]:
for p in pkls_paths:
    with open(p, 'rb') as f:
        dfs.append(pd.DataFrame(pickle.load(f)))
        names.append(p.split('/')[-1].split('.')[0])

In [6]:
names

['piercing',
 'pendant',
 'bracelet',
 'earrings',
 'necklace',
 'brooch',
 'ring',
 'anklet']

In [7]:
sum([d.shape[0] for d in dfs])

308421

In [8]:
dfs[0].shape

(651, 52)

In [9]:
dfs_copy = copy(dfs)

In [10]:
dt = dfs_copy.pop()
first_name = names.pop()
dt['jewelry_class'] = first_name

for df, name in zip(dfs, names):
    print(f"dt shape:{dt.shape}, df.shape{df.shape}")
    dt = dt.reset_index(drop=True)
    
    df['jewelry_class'] = name
    df = df.reset_index(drop=True)
    
    dt = pd.concat([dt, df], axis=0, sort=True)

dt shape:(42937, 53), df.shape(651, 52)
dt shape:(43588, 53), df.shape(15933, 52)
dt shape:(59521, 53), df.shape(49800, 52)
dt shape:(109321, 53), df.shape(49800, 52)
dt shape:(159121, 53), df.shape(49700, 52)
dt shape:(208821, 53), df.shape(49800, 52)
dt shape:(258621, 53), df.shape(49800, 52)


## Not good.

In [11]:
dt.isna().sum().sum()

2561026

In [12]:
dt.shape

(308421, 53)

In [13]:
dt.drop_duplicates('listing_id', inplace=True)

In [14]:
dt.shape

(308247, 53)

### Looking at which columns should drop, also referenced https://www.etsy.com/developers/documentation/reference/listing

In [15]:
dt.columns

Index(['category_id', 'category_path', 'category_path_ids', 'creation_tsz',
       'currency_code', 'description', 'ending_tsz', 'featured_rank',
       'file_data', 'has_variations', 'is_customizable', 'is_digital',
       'is_private', 'is_supply', 'is_vintage', 'item_dimensions_unit',
       'item_height', 'item_length', 'item_weight', 'item_weight_unit',
       'item_width', 'jewelry_class', 'language', 'last_modified_tsz',
       'listing_id', 'materials', 'non_taxable', 'num_favorers', 'occasion',
       'original_creation_tsz', 'price', 'processing_max', 'processing_min',
       'quantity', 'recipient', 'shipping_template_id', 'shop_section_id',
       'should_auto_renew', 'sku', 'state', 'state_tsz', 'style',
       'suggested_taxonomy_id', 'tags', 'taxonomy_id', 'taxonomy_path',
       'title', 'url', 'used_manufacturer', 'user_id', 'views', 'when_made',
       'who_made'],
      dtype='object')

In [16]:
drop_cols = ['category_id',
             'category_path_ids',
             'shipping_template_id',
             'shop_section_id',
             'suggested_taxonomy_id',
             'taxonomy_id',
             'url',
             'is_private',
            'file_data',
            'has_variations',
            'item_dimensions_unit',
            'item_length',
            'item_height',
            'item_weight',
            'item_weight_unit',
            'item_width',
            'should_auto_renew',
            'sku',
            'state',
            'style']

In [17]:
dt.drop(drop_cols, axis=1, inplace=True)

In [18]:
dt.shape

(308247, 33)

In [19]:
dt = dt[dt.language.isin(['en-US','MACHINE_en'])]
dt.drop(['language'], axis=1, inplace=True)

In [20]:
dt.occasion.isna().sum(), dt.description.isna().sum(), dt.processing_max.isna().sum(), dt.processing_min.isna().sum(), dt.title.isna().sum()

(297642, 0, 11911, 11911, 0)

In [21]:
#filling ints
dt.processing_max = dt.processing_max.fillna(0)
dt.processing_min = dt.processing_min.fillna(0)

#filling text
dt.occasion = dt.occasion.fillna('None')
dt.description = dt.description.fillna('None')
dt.title = dt.title.fillna('None')


In [22]:
dt.occasion.isna().sum(), dt.description.isna().sum(), dt.processing_max.isna().sum(), dt.processing_min.isna().sum(), dt.title.isna().sum()

(0, 0, 0, 0, 0)

In [23]:
dt.recipient.isna().sum()

250943

In [24]:
dt.recipient.unique()

array([None, 'children', 'women', 'not_specified', 'unisex_adults',
       'babies', 'teen_girls', 'men', 'girls', 'baby_girls', 'teens',
       'pets', 'cats', 'teen_boys', 'dogs', 'baby_boys', 'boys', 'birds'],
      dtype=object)

In [25]:
dt.recipient = dt.recipient.fillna('not_specified')

In [26]:
dt.recipient.isna().sum()

0

In [27]:
dt.who_made.unique()

array(['i_did', 'collective', 'someone_else', None], dtype=object)

In [28]:
dt.who_made = dt.who_made.fillna('someone_else')

In [29]:
dt.who_made.unique()

array(['i_did', 'collective', 'someone_else'], dtype=object)

### Feature Rank is only NaN when its so far away, so I wanna fill those with a high number to penalize

In [30]:
dt.featured_rank.describe()

count    10196.000000
mean        32.125245
std         96.416395
min          1.000000
25%          3.000000
50%          5.000000
75%         17.000000
max       1891.000000
Name: featured_rank, dtype: float64

In [31]:
max_recorded_rank = dt.featured_rank.describe()[-1]

In [32]:
dt.featured_rank.fillna(max_recorded_rank*2, inplace=True)

In [33]:
dt

,category_path,creation_tsz,currency_code,description,ending_tsz,featured_rank,is_customizable,is_digital,is_supply,is_vintage,...,recipient,state_tsz,tags,taxonomy_path,title,used_manufacturer,user_id,views,when_made,who_made
0,"[Jewelry, Anklet]",1.565354e+09,USD,This piece is just as beautiful in person as i...,1.575898e+09,3782.0,True,False,false,False,...,not_specified,1.565301e+09,"[Custom Card, Best Sister Gift, Swarovski Pear...","[Jewelry, Body Jewelry, Anklets]",Dark Gray Infinity Anklet - Dark Grey Pearl In...,False,32000761.0,82.0,made_to_order,i_did
1,"[Jewelry, Anklet]",1.565355e+09,USD,❤CHARMING ORGANZA FLOWERS AND PEARLS CHRISTENI...,1.575899e+09,3782.0,True,False,false,False,...,children,1.548678e+09,"[flower girl, baby barefoot, baby sandals, chr...","[Jewelry, Body Jewelry, Anklets]","Baby Barefoot Sandals, Toddler Jewelry,Newborn...",False,11175308.0,3601.0,made_to_order,i_did
2,"[Jewelry, Anklet]",1.565355e+09,USD,Shaky Shells Anklet. Made with round multi-co...,1.575899e+09,3782.0,False,False,false,False,...,not_specified,1.575395e+09,"[anklets, ankle bracelet, beaded anklet, chain...","[Jewelry, Body Jewelry, Anklets]",Shaky Shells Anklet,False,60381383.0,19.0,2010_2019,i_did
3,"[Jewelry, Anklet]",1.565355e+09,USD,"Garnet Hydro Anklet,Thin Delicate Anklet,India...",1.575899e+09,3782.0,False,False,false,False,...,not_specified,1.565355e+09,"[Handmade Jewelry, Fine Anklet, Boho Anklet, T...","[Jewelry, Body Jewelry, Anklets]","Garnet Hydro Anklet,Thin Delicate Anklet,India...",False,174209552.0,14.0,2010_2019,i_did
4,"[Jewelry, Anklet]",1.565356e+09,USD,This lovely bracelet is handcrafted with Swaro...,1.575900e+09,3782.0,False,False,false,False,...,not_specified,1.557072e+09,"[Swarovski crystal, sterling silver, birthston...","[Jewelry, Body Jewelry, Anklets]",Swarovski Crystal and Sterling Silver Bracelet,False,6096032.0,37.0,2010_2019,i_did
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49795,"[Jewelry, Ring]",1.565771e+09,USD,"Oval Moissanite Ring, Oval Cut 7x9mm Moissanit...",1.576316e+09,3782.0,False,False,false,False,...,not_specified,1.544606e+09,"[moissanite ring, art deco ring, engagement ri...","[Jewelry, Rings, Wedding & Engagement, Engagem...","Oval Moissanite Ring, Oval Cut 7x9mm Moissanit...",False,138745551.0,219.0,made_to_order,collective
49796,"[Jewelry, Ring]",1.565771e+09,EUR,Design ring made in my parisian workshop!\nPie...,1.576316e+09,3782.0,False,False,false,False,...,not_specified,1.523042e+09,"[SILVER, ring, women, men, ligns, comb, design...","[Jewelry, Rings]",CO2,False,24212557.0,168.0,2010_2019,i_did
49797,"[Jewelry, Ring]",1.565771e+09,USD,Welcome to TampicoJewelry.com!\nVictorian Inta...,1.576316e+09,3782.0,True,False,false,True,...,not_specified,1.522793e+09,"[silver signet ring, intaglio ring, jasper int...","[Jewelry, Rings, Signet Rings]","Victorian Intaglio, c1900, Wax Seal Ring, Engr...",False,24003253.0,1180.0,1980s,someone_else
49798,"[Jewelry, Ring]",1.565771e+09,EUR,Silver ring 925 gold rose gold (3 microns) set...,1.576316e+09,3782.0,False,False,false,False,...,not_specified,1.565772e+09,"[jewelry, Woman, ring, Stone, Lonely, Amethyst...","[Jewelry, Rings, Multi-Stone Rings]",Ring Solitaire silver gilded rose gold and lav...,False,94944764.0,198.0,2010_2019,someone_else


In [34]:
def cols_to_vec(l):
    try:
        return ', '.join(l)
    except TypeError:
        return ''

In [35]:
dt.category_path = dt.category_path.apply(cols_to_vec)
dt.materials = dt.materials.apply(cols_to_vec)
dt.tags = dt.tags.apply(cols_to_vec)
dt.taxonomy_path = dt.taxonomy_path.apply(cols_to_vec)

dt.head()

,category_path,creation_tsz,currency_code,description,ending_tsz,featured_rank,is_customizable,is_digital,is_supply,is_vintage,...,recipient,state_tsz,tags,taxonomy_path,title,used_manufacturer,user_id,views,when_made,who_made
0,"Jewelry, Anklet",1.565354e+09,USD,This piece is just as beautiful in person as i...,1.575898e+09,3782.0,True,False,false,False,...,not_specified,1.565301e+09,"Custom Card, Best Sister Gift, Swarovski Pearl...","Jewelry, Body Jewelry, Anklets",Dark Gray Infinity Anklet - Dark Grey Pearl In...,False,32000761.0,82.0,made_to_order,i_did
1,"Jewelry, Anklet",1.565355e+09,USD,❤CHARMING ORGANZA FLOWERS AND PEARLS CHRISTENI...,1.575899e+09,3782.0,True,False,false,False,...,children,1.548678e+09,"flower girl, baby barefoot, baby sandals, chri...","Jewelry, Body Jewelry, Anklets","Baby Barefoot Sandals, Toddler Jewelry,Newborn...",False,11175308.0,3601.0,made_to_order,i_did
2,"Jewelry, Anklet",1.565355e+09,USD,Shaky Shells Anklet. Made with round multi-co...,1.575899e+09,3782.0,False,False,false,False,...,not_specified,1.575395e+09,"anklets, ankle bracelet, beaded anklet, chain ...","Jewelry, Body Jewelry, Anklets",Shaky Shells Anklet,False,60381383.0,19.0,2010_2019,i_did
3,"Jewelry, Anklet",1.565355e+09,USD,"Garnet Hydro Anklet,Thin Delicate Anklet,India...",1.575899e+09,3782.0,False,False,false,False,...,not_specified,1.565355e+09,"Handmade Jewelry, Fine Anklet, Boho Anklet, Tr...","Jewelry, Body Jewelry, Anklets","Garnet Hydro Anklet,Thin Delicate Anklet,India...",False,174209552.0,14.0,2010_2019,i_did
4,"Jewelry, Anklet",1.565356e+09,USD,This lovely bracelet is handcrafted with Swaro...,1.575900e+09,3782.0,False,False,false,False,...,not_specified,1.557072e+09,"Swarovski crystal, sterling silver, birthstone...","Jewelry, Body Jewelry, Anklets",Swarovski Crystal and Sterling Silver Bracelet,False,6096032.0,37.0,2010_2019,i_did


In [36]:
dt.category_path.value_counts().head()

Jewelry, Brooch      49494
Jewelry, Ring        49291
Jewelry, Necklace    49002
Jewelry, Earrings    48943
Jewelry, Bracelet    48916
Name: category_path, dtype: int64

In [37]:
dt.materials.value_counts().head()

          55863
Silver    19072
Glass      6005
Gold       4466
Brass      3043
Name: materials, dtype: int64

In [38]:
dt.tags.value_counts().head()

                                                                                                         17562
Anzor, Russian, Jewelry, Imperial, Elegant, Diamond, Bride, Victorian, Multitone, Rose, White, NYC         405
jewelry, earrings                                                                                          262
earrings                                                                                                   195
ring, keys, pin, metal, dog, dog lover, collection, gift, souvenir, stylish, exclusive, show, Buttons      163
Name: tags, dtype: int64

In [39]:
dt.taxonomy_path.value_counts().head()

Jewelry, Brooches, Pins & Clips, Brooches    49755
Jewelry, Body Jewelry, Anklets               42828
Jewelry, Necklaces, Pendants                 36809
Jewelry, Earrings, Dangle & Drop Earrings    28542
Jewelry, Bracelets, Beaded Bracelets         18928
Name: taxonomy_path, dtype: int64

In [40]:
dt.when_made.value_counts()

2010_2019        171567
made_to_order     69909
before_2000       22289
1960s              8382
1980s              8101
1970s              6940
1950s              6008
1940s              3055
1990s              2750
2000_2009          2039
1930s              1824
1800s              1562
1920s              1382
1900s              1003
1910s               706
False                98
before_1700          53
1700s                39
Name: when_made, dtype: int64

In [41]:
def filter_made(era):
    if era == 'made_to_order':
        return era
    
    elif era in ['2010_2019', '2000_2009']:
        return 'modern'
    
    else:
        return 'old'

In [42]:
dt.when_made = dt.when_made.apply(filter_made)

In [43]:
dt.when_made.value_counts()

modern           173606
made_to_order     69909
old               64392
Name: when_made, dtype: int64

#### Currencies

In [44]:
dt.currency_code.value_counts()

USD    235281
EUR     31989
GBP     24341
CAD      8884
AUD      5385
ILS       607
SEK       275
SGD       230
CHF       165
NZD       149
HKD       142
THB       140
JPY        99
DKK        80
NOK        72
MXN        45
TWD        11
PLN         7
PHP         5
Name: currency_code, dtype: int64

In [45]:
top_k = 5
top_curr = list(dt.currency_code.value_counts())[:top_k]
f"The top {top_k} currencies hold {round((sum(top_curr) - sum(list(dt.currency_code.value_counts())[5:]))/dt.shape[0], 2)}% of the entries, so we are gonna drop the rest"

'The top 5 currencies hold 0.99% of the entries, so we are gonna drop the rest'

In [46]:
top_cur = ['USD', 'EUR','GBP','CAD','AUD']
top_cur

['USD', 'EUR', 'GBP', 'CAD', 'AUD']

In [47]:
top_cur = ['USD']

In [48]:
dt = copy(dt[dt['currency_code'].isin(top_cur)])

### Occasions

In [49]:
dt.occasion.value_counts()

None                  226292
birthday                1930
wedding                 1527
christmas               1063
anniversary             1061
engagement               822
valentines               555
mothers_day              481
get_well                 187
new_years                170
sympathy                 168
halloween                155
graduation               100
sweet_16                  93
new_baby                  82
day_of_the_dead           74
fathers_day               72
easter                    62
housewarming              53
chinese_new_year          47
confirmation              40
st_patricks_day           38
july_4th                  38
thanksgiving              32
eid                       31
prom                      27
baptism                   21
canada_day                18
hanukkah                  13
bar_or_bat_mitzvah        13
cinco_de_mayo              7
kwanzaa                    4
quinceanera                3
retirement                 2
Name: occasion

In [50]:
def occasion_filter(oc):
    top_categories = ['None', 'birthday','wedding',
                      'anniversary','christmas','engagement',
                      'valentines','mothers_day','sympathy',
                      'get_well','new_years','halloween']
    if oc in top_categories:
        return oc
    else:
        return 'other'

In [51]:
dt.occasion = dt.occasion.apply(lambda x: occasion_filter(x))

In [52]:
dt.occasion.value_counts()

None           226292
birthday         1930
wedding          1527
christmas        1063
anniversary      1061
other             870
engagement        822
valentines        555
mothers_day       481
get_well          187
new_years         170
sympathy          168
halloween         155
Name: occasion, dtype: int64

#### Boolean fields seem to have nans and False for the same thing. I will also convert them to their approperate int value for the 

In [53]:
cols = list(dt.columns)
bool_cols = [c if 'is_' in c else False for c in cols]

other_cols = ['used_manufacturer']
bool_cols.extend(other_cols)

assert len(bool_cols) == len(cols) + len(other_cols)

bool_cols = list(filter(None, bool_cols))

In [54]:
bool_cols

['is_customizable',
 'is_digital',
 'is_supply',
 'is_vintage',
 'used_manufacturer']

In [55]:
dt[bool_cols] = dt[bool_cols].fillna(False)

In [56]:
dt[bool_cols[0]].unique()

array([ True, False])

In [57]:
[print(x) for x in dt[bool_cols].iloc[0]]

True
False
false
False
False


[None, None, None, None, None]

In [58]:
int(bool(dt[bool_cols].iloc[0][2]))

1

In [59]:
# dt[bool_cols].iloc[0].apply(lambda row: [int(el) for el in row]) 
# need to convert to bool first bc some are type str 

dt[bool_cols] = dt[bool_cols].apply(lambda row: [int(bool(el)) for el in row])

In [60]:
dt.head()

,category_path,creation_tsz,currency_code,description,ending_tsz,featured_rank,is_customizable,is_digital,is_supply,is_vintage,...,recipient,state_tsz,tags,taxonomy_path,title,used_manufacturer,user_id,views,when_made,who_made
0,"Jewelry, Anklet",1.565354e+09,USD,This piece is just as beautiful in person as i...,1.575898e+09,3782.0,1,0,1,0,...,not_specified,1.565301e+09,"Custom Card, Best Sister Gift, Swarovski Pearl...","Jewelry, Body Jewelry, Anklets",Dark Gray Infinity Anklet - Dark Grey Pearl In...,0,32000761.0,82.0,made_to_order,i_did
1,"Jewelry, Anklet",1.565355e+09,USD,❤CHARMING ORGANZA FLOWERS AND PEARLS CHRISTENI...,1.575899e+09,3782.0,1,0,1,0,...,children,1.548678e+09,"flower girl, baby barefoot, baby sandals, chri...","Jewelry, Body Jewelry, Anklets","Baby Barefoot Sandals, Toddler Jewelry,Newborn...",0,11175308.0,3601.0,made_to_order,i_did
2,"Jewelry, Anklet",1.565355e+09,USD,Shaky Shells Anklet. Made with round multi-co...,1.575899e+09,3782.0,0,0,1,0,...,not_specified,1.575395e+09,"anklets, ankle bracelet, beaded anklet, chain ...","Jewelry, Body Jewelry, Anklets",Shaky Shells Anklet,0,60381383.0,19.0,modern,i_did
3,"Jewelry, Anklet",1.565355e+09,USD,"Garnet Hydro Anklet,Thin Delicate Anklet,India...",1.575899e+09,3782.0,0,0,1,0,...,not_specified,1.565355e+09,"Handmade Jewelry, Fine Anklet, Boho Anklet, Tr...","Jewelry, Body Jewelry, Anklets","Garnet Hydro Anklet,Thin Delicate Anklet,India...",0,174209552.0,14.0,modern,i_did
4,"Jewelry, Anklet",1.565356e+09,USD,This lovely bracelet is handcrafted with Swaro...,1.575900e+09,3782.0,0,0,1,0,...,not_specified,1.557072e+09,"Swarovski crystal, sterling silver, birthstone...","Jewelry, Body Jewelry, Anklets",Swarovski Crystal and Sterling Silver Bracelet,0,6096032.0,37.0,modern,i_did


### Better

In [61]:
dt.isna().sum().sum()

3

In [62]:
dt.dropna(inplace=True)

### Best

In [63]:
dt.isna().sum().sum()

0

In [64]:
dt.drop(['currency_code'], axis=1, inplace=True)

In [65]:
dt.to_pickle('datsets/jewelry_all.pkl')

In [66]:
classes = dt.jewelry_class.unique()

dt.to_pickle('datsets/jewelry_all.pkl')

classes = dt.jewelry_class.unique()


for c in classes:
    dx = copy(dt[dt['jewelry_class'] == c])
    dx.drop(['jewelry_class'], axis=1, inplace=True)
    path= f'datsets/{c}.pkl'
    dt.to_pickle(path)
    print(f'datsets/{c}.pkl')

datsets/anklet.pkl
datsets/piercing.pkl
datsets/pendant.pkl
datsets/bracelet.pkl
datsets/earrings.pkl
datsets/necklace.pkl
datsets/brooch.pkl
datsets/ring.pkl


# Model

In [67]:
dt['has_silver'] = dt.materials.str.contains('silver', case = False, regex=False).apply(lambda x:1 if x else 0)
dt['has_gold'] = dt.materials.str.contains('gold', case = False, regex=False).apply(lambda x:1 if x else 0)
dt['has_glass'] = dt.materials.str.contains('glass', case = False, regex=False).apply(lambda x:1 if x else 0)
dt['has_diamond'] = dt.materials.str.contains('diamond', case = False, regex=False).apply(lambda x:1 if x else 0)
dt['has_pearl'] = dt.materials.str.contains('pearl', case = False, regex=False).apply(lambda x:1 if x else 0)
stones = r'sapphire|quartz|ruby|turquoise||amethyst|stone|gemstone|emerald'
dt['has_stone'] = dt.materials.str.contains(stones, case = False, regex=True).apply(lambda x:1 if x else 0)

In [68]:
dt

,category_path,creation_tsz,description,ending_tsz,featured_rank,is_customizable,is_digital,is_supply,is_vintage,jewelry_class,...,user_id,views,when_made,who_made,has_silver,has_gold,has_glass,has_diamond,has_pearl,has_stone
0,"Jewelry, Anklet",1.565354e+09,This piece is just as beautiful in person as i...,1.575898e+09,3782.0,1,0,1,0,anklet,...,32000761.0,82.0,made_to_order,i_did,1,0,0,0,1,1
1,"Jewelry, Anklet",1.565355e+09,❤CHARMING ORGANZA FLOWERS AND PEARLS CHRISTENI...,1.575899e+09,3782.0,1,0,1,0,anklet,...,11175308.0,3601.0,made_to_order,i_did,0,0,0,0,1,1
2,"Jewelry, Anklet",1.565355e+09,Shaky Shells Anklet. Made with round multi-co...,1.575899e+09,3782.0,0,0,1,0,anklet,...,60381383.0,19.0,modern,i_did,1,0,0,0,0,1
3,"Jewelry, Anklet",1.565355e+09,"Garnet Hydro Anklet,Thin Delicate Anklet,India...",1.575899e+09,3782.0,0,0,1,0,anklet,...,174209552.0,14.0,modern,i_did,1,0,0,0,0,1
4,"Jewelry, Anklet",1.565356e+09,This lovely bracelet is handcrafted with Swaro...,1.575900e+09,3782.0,0,0,1,0,anklet,...,6096032.0,37.0,modern,i_did,1,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49788,"Jewelry, Ring",1.565771e+09,The ring was made hand made of 925 silver and ...,1.576316e+09,6.0,0,0,1,0,ring,...,7934612.0,97.0,modern,i_did,1,0,0,0,0,1
49789,"Jewelry, Ring",1.565771e+09,Emerald Ring Gold Fill 14K Ring \nMetal Type -...,1.576316e+09,3782.0,1,0,1,0,ring,...,46771602.0,4734.0,made_to_order,i_did,0,1,0,0,0,1
49790,"Jewelry, Ring",1.565771e+09,When nature is designed as a wedding decoratio...,1.576316e+09,3782.0,1,0,1,0,ring,...,78465143.0,62.0,made_to_order,i_did,0,1,0,1,0,1
49795,"Jewelry, Ring",1.565771e+09,"Oval Moissanite Ring, Oval Cut 7x9mm Moissanit...",1.576316e+09,3782.0,0,0,1,0,ring,...,138745551.0,219.0,made_to_order,collective,0,1,0,0,0,1
